In [1]:
%load_ext autoreload
%autoreload 2
!git clone --branch=add-data https://github.com/schmidt-jake/kaggle.git
%cd kaggle
!pip install -U pip setuptools wheel
!pip install -U -r /kaggle/working/kaggle/requirements/main.in

Cloning into 'kaggle'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 209 (delta 95), reused 176 (delta 66), pack-reused 0
Receiving objects: 100% (209/209), 35.25 KiB | 3.52 MiB/s, done.
Resolving deltas: 100% (95/95), done.
/kaggle/working/kaggle
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.1 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
  Attempting uninstall: pip
    Found existing installation: pip 22.1.1
    Uninstalling pip-22.1.1:
      Successfully uninstalled pip-22.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependen

In [17]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 406 bytes | 406.00 KiB/s, done.
From https://github.com/schmidt-jake/kaggle
   e025969..bb2f927  add-data   -> origin/add-data
Updating e025969..bb2f927
Fast-forward
 mayo_clinic_strip_ai/dataset.py | 5 +++--
 1 file changed, 3 insertions(+), 2 deletions(-)


In [ ]:
from glob import glob

import pandas as pd
import torch
from torch.utils.data import DataLoader
from torchmetrics import MetricCollection
from torchmetrics.classification import Accuracy
from torchmetrics.classification import CalibrationError
from torchvision.models import densenet161

from mayo_clinic_strip_ai.dataset import TifDataset
from mayo_clinic_strip_ai.model import Classifier
from mayo_clinic_strip_ai.model import FeatureExtractor
from mayo_clinic_strip_ai.model import Loss
from mayo_clinic_strip_ai.model import Model
from mayo_clinic_strip_ai.model import Normalizer
from mayo_clinic_strip_ai.metadata import load_metadata


train_meta = pd.merge(
    left=pd.read_csv("/kaggle/input/mayo-rois/train/ROIs.csv"),
    right=load_metadata("/kaggle/input/mayo-clinic-strip-ai/train.csv"),
    how="left",
    validate="m:1",
    on="image_id",
)

model = Model(
    normalizer=Normalizer(),
    feature_extractor=FeatureExtractor(backbone=densenet161()),
    classifier=Classifier(initial_logit_bias=0.0, in_features=2208),  # FIXME: auto-set in_features
)
optimizer = torch.optim.AdamW(model.parameters())
loss_fn = Loss(pos_weight=0.5)  # FIXME: compute pos_weight from data
metrics = MetricCollection(Accuracy(), CalibrationError())

# move things to the right device
device = torch.device("cuda", 0) if torch.cuda.is_available() else torch.device("cpu")
model.to(device=device, memory_format=torch.channels_last)
loss_fn.to(device=device)
metrics.to(device=device)

train_dataset = TifDataset(
    metadata=train_meta,
    training=True,
    data_dir="/kaggle/input/mayo-clinic-strip-ai/train/",
)
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=2,
    shuffle=True,
    pin_memory=torch.cuda.is_available(),
)

grad_scaler = torch.cuda.amp.GradScaler()

for epoch in range(10):
    for img, label_id in train_dataloader:
        img = img.to(device=device)
        label_id = label_id.to(device=device)
        with torch.autocast(device_type=img.device.type):
            optimizer.zero_grad(set_to_none=True)
            logit: torch.Tensor = model(img)
            loss: torch.Tensor = loss_fn(logit=logit, label=label_id)
            grad_scaler.scale(loss).backward()
            grad_scaler.step(optimizer)
            grad_scaler.update()
            metrics.update(preds=logit.sigmoid(), target=label_id)
        print(metrics.compute())

/opt/conda/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
